In [ ]:
# Folder containing the CSV files
folder_path = r"https://drive.google.com/drive/folders/154dzcj0f7qXlNbhPp0luRj0ORpcxZ_FB?usp=drive_link"

# List of CSV files and their corresponding table names
csv_files = [
    ("customers.csv", "customers"),
    ("orders.csv", "orders")
]


In [ ]:
import pandas as pd
import psycopg2
import os
import glob

# PostgreSQL connection
conn = psycopg2.connect(
    host='localhost',
    port='5432',
    user='postgres',
    password='Anika@2003',
    dbname='ecommerce_analytics'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = r"https://drive.google.com/drive/folders/154dzcj0f7qXlNbhPp0luRj0ORpcxZ_FB?usp=drive_link"

# Map pandas dtype to PostgreSQL data type
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INTEGER'
    elif pd.api.types.is_float_dtype(dtype):
        return 'DOUBLE PRECISION'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'TIMESTAMP'
    else:
        return 'TEXT'

# Find all CSV files in folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

print("Found CSV files:", csv_files)

for file_path in csv_files:
    # Use filename (without extension) as table name
    file_name = os.path.basename(file_path)
    table_name = os.path.splitext(file_name)[0].lower().replace(" ", "_")

    df = pd.read_csv(file_path)
    df = df.where(pd.notnull(df), None)

    print(f"\nProcessing {file_name} → Table: {table_name}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [
        col.strip().replace(' ', '_').replace('-', '_').replace('.', '_')
        for col in df.columns
    ]

    # Create table if not exists
    columns = ', '.join([f'"{col}" {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS "{table_name}" ({columns});'
    cursor.execute(create_table_query)

    # Prepare insert statement
    col_names = ', '.join([f'"{col}"' for col in df.columns])
    placeholders = ', '.join(['%s'] * len(df.columns))
    insert_query = f'INSERT INTO "{table_name}" ({col_names}) VALUES ({placeholders})'

    # Insert rows efficiently
    values = [
        tuple(None if pd.isna(x) else x for x in row)
        for row in df.itertuples(index=False, name=None)
    ]
    cursor.executemany(insert_query, values)

    conn.commit()
    print(f"Inserted {len(values)} rows into {table_name}")

# Close connections
cursor.close()
conn.close()


Found CSV files: ['C:\\Users\\91852\\OneDrive\\Desktop\\ecommerce analysis\\customers (2).csv', 'C:\\Users\\91852\\OneDrive\\Desktop\\ecommerce analysis\\orders (1).csv']

Processing customers (2).csv → Table: customers_(2)
NaN values before replacement:
CustomerID      0
Name            0
Age             0
Gender          0
City            0
LoyaltyScore    0
dtype: int64

Inserted 200 rows into customers_(2)

Processing orders (1).csv → Table: orders_(1)
NaN values before replacement:
OrderID            0
CustomerID         0
OrderDate          0
ShipDate           0
Region             0
ProductCategory    0
ProductName        0
Quantity           0
UnitPrice          0
PaymentMode        0
Status             0
dtype: int64

Inserted 1000 rows into orders_(1)


In [ ]:
import pandas as pd
import psycopg2
import os
import glob

# PostgreSQL connection
db = psycopg2.connect(
    host='localhost',
    port='5432',
    user='postgres',
    password='Anika@2003',
    dbname='ecommerce_analytics'
)
cursor = db.cursor()

# (a) Revenue by Region

In [ ]:
query="""select region, sum(quantity*unitprice)as region_revenue
from orders
group by region
order by region_revenue desc"""

cursor.execute(query)

data=cursor.fetchall()
df=pd.DataFrame(data, columns=["region", "region_revenue"])
df


,region,region_revenue
0,West,20890216.00
1,East,19584357.00
2,North,18387044.00
3,South,17543421.00


# (b) Top 10 Customers by Lifetime Value

In [ ]:
query = """
SELECT c.name, SUM(o.quantity * o.unitprice) AS total_revenue
FROM customers c
JOIN orders o ON c.customerID = o.customerID
GROUP BY c.name
ORDER BY total_revenue DESC
limit 10;
"""

cursor.execute(query)

data = cursor.fetchall()
df = pd.DataFrame(data, columns=["name", "total_revenue"])
df


,name,total_revenue
0,Ankit,11045345.00
1,Priya,10059858.00
2,Neha,9663625.00
3,Amit,9182649.00
4,Pooja,8555721.00
5,Aarav,6223453.00
6,Kavya,5732785.00
7,Rohan,5406422.00
8,Simran,5331803.00
9,Vikram,5203377.00


# (c) Best-Selling Product Categories

In [ ]:
query = """
select productcategory, sum(quantity*unitprice)as total_revenue
from orders
group by  productcategory
order by total_revenue;
"""

cursor.execute(query)

data = cursor.fetchall()
df = pd.DataFrame(data, columns=["productcategory", "total_revenue"])
df


,productcategory,total_revenue
0,Furniture,13201238.00
1,Fashion,14335041.00
2,Groceries,15553810.00
3,Electronics,15755021.00
4,Sports,17559928.00


# (d) Delay Impact on Revenue

In [ ]:
query = """
select status,
COUNT(*) AS order_count,
sum(quantity*unitprice)as total_revenue
from orders
group by status
order by total_revenue desc
"""

cursor.execute(query)

data = cursor.fetchall()
df = pd.DataFrame(data, columns=["status","order_count", "total_revenue"])
df


,status,order_count,total_revenue
0,Shipped,729,56375110.00
1,Delayed,180,14167356.00
2,Returned,91,5862572.00


# (e) Monthly Revenue Trend

In [ ]:
query = """
SELECT
TO_CHAR(orderdate, 'YYYY-MM') AS months,
SUM(quantity * unitprice) AS monthaly_revenue
FROM orders
GROUP BY TO_CHAR(orderdate, 'YYYY-MM')
ORDER BY monthaly_revenue DESC;
"""

cursor.execute(query)

data = cursor.fetchall()
df = pd.DataFrame(data, columns=["months", "monthaly_revenue"])
df


,months,monthaly_revenue
0,2023-03,7226989.00
1,2023-09,7188058.00
2,2023-05,7146873.00
3,2023-06,7032110.00
4,2023-12,6805319.00
5,2023-07,6541976.00
6,2023-02,6301346.00
7,2023-01,6262854.00
8,2023-04,5808776.00
9,2023-11,5721601.00
